# Computed Fields

Sometimes, we want those properties to be actual fields in our Pydantic model - which means they get serialized just like any other field.

To do that Pydantic provides us the `@computed_field` decorator. 

In [1]:
from functools import cached_property
from math import pi
from pydantic import BaseModel, computed_field, Field, PydanticUserError, ValidationError

Let's use our `Circle` example again:

In [2]:
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0, frozen=True)

    @property
    def area(self):
        print("calculating area...")
        return pi * self.radius ** 2    

All we need to do to make that property a field in our model (albeit a computed field), is to decorate it. However, since this is now a field in a Pydantic model, we **MUST** set a type hint on the property return value so taht Pydantic knows what the type for that field is.

If you forget that, you'll get a `PydanticUserError` exception:

In [3]:
try:
    class Circle(BaseModel):
        center: tuple[int, int] = (0, 0)
        radius: int = Field(default=1, gt=0, frozen=True)
    
        @computed_field
        @property
        def area(self):
            print("calculating area...")
            return pi * self.radius ** 2    
except PydanticUserError as ex:
    print(ex)

Computed field is missing return type annotation or specifying `return_type` to the `@computed_field` decorator (e.g. `@computed_field(return_type=int|str)`)

For further information visit https://errors.pydantic.dev/2.5/u/model-field-missing-annotation


As the exception notes, we can either specify it as a type hint, or as an argument in the decorator - the more usual way is to use the type hint:

In [4]:
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0, frozen=True)

    @computed_field
    @property
    def area(self) -> float:
        print("calculating area...")
        return pi * self.radius ** 2  

And now, when we create a model instance:

In [5]:
c = Circle()

In [6]:
c

calculating area...


Circle(center=(0, 0), radius=1, area=3.141592653589793)

As you can see, `area` is now in the representation, and is also in the serialized data:

In [7]:
c.model_dump()

calculating area...


{'center': (0, 0), 'radius': 1, 'area': 3.141592653589793}

In [8]:
c.model_dump_json()

calculating area...


'{"center":[0,0],"radius":1,"area":3.141592653589793}'

First thing is we may want to give it an alias (probably not in this case, but in general you can):

In [9]:
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0, frozen=True)

    @computed_field(alias="AREA")
    @property
    def area(self) -> float:
        print("calculating area...")
        return pi * self.radius ** 2  

In [10]:
c = Circle()
c.model_dump(by_alias=True)

calculating area...


{'center': (0, 0), 'radius': 1, 'AREA': 3.141592653589793}

I don't know if I showed you this before, but we can choose to omit certain fields from the instance representation:

In [11]:
class Model(BaseModel):
    field_1: int = Field(repr=False)
    field_2: int

In [12]:
m = Model(field_1=1, field_2=2)
m

Model(field_2=2)

As you can see, `field_1` does not show up in the representation, but it is still a regular field, and can be accessed via dot notation, and gets serialized as usual:

In [13]:
m.field_1=10
m.field_1

10

In [14]:
m.model_dump()

{'field_1': 10, 'field_2': 2}

Now, with these computed properties, we do not have that `Field` mechanism to omit a computed field from the representation. But the `@computed_field` decorator, does enable that option, also using `repr` as an argument:

In [15]:
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0, frozen=True)

    @computed_field(alias="AREA", repr=False)
    @property
    def area(self) -> float:
        print("calculating area...")
        return pi * self.radius ** 2  

In [16]:
c = Circle()

In [17]:
c

Circle(center=(0, 0), radius=1)

As you can see, `area` is not in the representation, but is accessible as usual:

In [18]:
c.area

calculating area...


3.141592653589793

and serializes as normal:

In [19]:
c.model_dump()

calculating area...


{'center': (0, 0), 'radius': 1, 'area': 3.141592653589793}

Note that a calculated field is read-only:

In [20]:
try:
    c.area = 10
except AttributeError as ex:
    print(ex)

property 'area' of 'Circle' object has no setter


Technically, Pydantic deos support the creation of a setter for a computed field - but given that I fail to see a realistic use case for it, I won't cover it here.

It is relatively straightforward, and if you do happen to need this, it is documented [here](https://docs.pydantic.dev/2.0/usage/computed_fields/)

Computed fields work with cached properties too:

In [21]:
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0, frozen=True)

    @computed_field(alias="AREA", repr=False)
    @cached_property
    def area(self) -> float:
        print("calculating area...")
        return pi * self.radius ** 2  

In [22]:
c = Circle()
c.area

calculating area...


3.141592653589793

In [23]:
c.area

3.141592653589793

As you can see, caching took effect, and second call did not actually run the computations for the area.